# Setup

 Import standard modules

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
os.chdir('/home/chris/projects/tidal_flat_0d')

In [2]:
from src import features, models
from pyprojroot import here
root = here()
sys.path.append(str(root))
wdir = root / "data" / "interim"

Import project modules from src folder

In [196]:
tides = features.load_tide(wdir, "tides.feather")
tides = tides - (np.mean(tides) + 0.6)
# tides = tides.iloc[:int(len(tides) * 0.35)]

In [261]:
sim.pbar.n

15

In [264]:
from src import models

sim = models.Simulation(
    tide_ts=tides,
    land_elev_init=1.15,
    conc_init=0.0,
    conc_bound=1.0,
    grain_diam=3.5e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate=2e-4,
    comp_rate=-4e-3,
    sub_rate=-3e-3,
    slr=0.005,
    solver_method="DOP853",
    runs=1,
    min_depth=0.0015,
    dense_output=False,
)
sim.run_one()

100%|██████████| 185/185.0 [04:28<00:00,  1.38s/it]

In [ ]:
import time
def aggrade_fd(times, tide_fun, init_land_elev, bound_conc, settle_rate, bulk_dens,):
    start_time = time.perf_counter()
    land_elev = np.array([init_land_elev, init_land_elev])
    conc = np.array([0.0, 0.0])
    depth = np.array([tide_fun(0) - init_land_elev])
    status = 0
    index = np.arange(0, len(times))
    dt = times[1] - times[0]
    for i in index[1:-1]:
        t = times[i]
        depth = np.append(depth, tide_fun(t) - land_elev[i])
        if depth[i] < 0.0015:
            conc = np.append(conc, 0.0)
            land_elev = np.append(land_elev, land_elev[-1])
            continue

        if (depth[i] - depth[i-1]) > 0:
            delta_conc = - settle_rate * conc[i] / depth[i] * dt - 1 / depth[i] * (conc[i] - bound_conc) * (depth[i] - depth[i-1])
        elif (depth[i] - depth[i-1]) <= 0:
            if status == 0:
                slack = times[i]
                status = 1
            delta_conc = - settle_rate * conc[i] / depth[i] * dt
            
        if delta_conc < 0 and abs(delta_conc) > conc[i]:
            print("Extracted too much")
            delta_conc = - conc[i]
            
        conc = np.append(conc, conc[i] + delta_conc)
        delta_land_elev = settle_rate * conc[i] / bulk_dens * dt
        land_elev = np.append(land_elev, land_elev[i] + delta_land_elev)
        
    depth = np.append(depth, tide_fun(i+1) - land_elev[i])
    print("Aggradation:           {:.2e} m".format(land_elev[-1] - land_elev[0]))
    print("Degradation:           {:.2e} m".format(0.0))
    print("Elevation change:      {:.2e} m".format(land_elev[-1] - land_elev[0]))
    print("Final elevation:       {:.4f} m".format(land_elev[-1]))
    print("Runtime:               {}".format(time.strftime("%H:%M:%S", time.gmtime(time.perf_counter() - start_time))))

    return slack, np.stack((times, depth, conc, land_elev))